## 데이터 로드

In [1]:
import json
import pandas as pd
import math
import numpy as np
from scipy import stats
import multiprocessing
import requests

In [2]:
with open('../Datasets/train_datas.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [3]:
train_data = pd.DataFrame(json_data)
#train_data = train_data.drop(['id', 'plylst_title', 'updt_date'], axis=1)
train_data.head()

,tags,song_id,like_cnt
0,"[미세먼지, 여유로운, 출근길, 아침, 잔잔한, 드라이브, 감성, 기분전환, 스트레...","[116805, 366786, 521739, 212014, 36179, 672550...",5
1,"[힙합, 힙합엘이, 블랙뮤직, HIPHOPLE, 흑인음악]","[406498, 271311, 123597, 699012, 137952, 70319...",0
2,"[여름, 기분전환]","[606905, 91361, 215618, 341999, 618007, 389726...",2
3,"[잔잔, 새벽, 카페, loveandlyrics, moody]","[495532, 677144, 34898, 657439, 525848, 386995...",6
4,"[홍대, 인디음악, Rock, 인디밴드, 상수, 참이슬, 인디]","[565726, 69242, 613056, 465889, 522895, 141820...",6


In [4]:
with open('../Datasets/song_meta_with_likes.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [5]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id,like_cnt_song
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0,0.616807
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1,0.000000
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2,0.000000
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3,22.996574
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4,0.297234


## 데이터 열 이름 변경

In [6]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id,like_cnt
0,"[미세먼지, 여유로운, 출근길, 아침, 잔잔한, 드라이브, 감성, 기분전환, 스트레...","[116805, 366786, 521739, 212014, 36179, 672550...",5
1,"[힙합, 힙합엘이, 블랙뮤직, HIPHOPLE, 흑인음악]","[406498, 271311, 123597, 699012, 137952, 70319...",0
2,"[여름, 기분전환]","[606905, 91361, 215618, 341999, 618007, 389726...",2
3,"[잔잔, 새벽, 카페, loveandlyrics, moody]","[495532, 677144, 34898, 657439, 525848, 386995...",6
4,"[홍대, 인디음악, Rock, 인디밴드, 상수, 참이슬, 인디]","[565726, 69242, 613056, 465889, 522895, 141820...",6


## 데이터 추출

- 500개의 플레이리스트 추출

In [7]:
#train_data_sample = train_data[:500].copy()
train_data_origin = train_data.sample(n=300)
train_data_indices = train_data_origin.index # 영어 전처리 시, 동일한 인덱스의 인스턴스를 가져오기 위함
train_data_origin = train_data_origin.reset_index(drop=True)
train_data_sample = train_data_origin.copy()
train_data_origin.shape
#train_data_sample = train_data

(300, 3)

In [8]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id,like_cnt_song
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0,0.616807
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1,0.000000
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2,0.000000
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3,22.996574
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4,0.297234


## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [9]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head(30)

,tags,song_id,like_cnt
0,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",314824,10
1,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",249789,10
2,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",28215,10
3,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",497515,10
4,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",650192,10
5,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",182422,10
6,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",503210,10
7,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",635003,10
8,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",78009,10
9,"[햇살, 카페, 여유, 오후, 재즈, 나른한, 커피, 감성, 향긋한]",393368,10


In [10]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)

In [11]:
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(10636, 3)

In [12]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

C:\Users\Hyun\AppData\Local\Temp\ipykernel_15164\1966443478.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_sample['tags'].iloc[i] = list(train_dict[song])


,tags,song_id,like_cnt
0,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",314824,10
1,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",249789,10
2,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",28215,10
3,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",497515,10
4,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",650192,10


In [13]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,like_cnt,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,like_cnt_song
0,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",314824,10,"[GN1912, GN1904, GN1901]",20070914,534654,"[408062, 408064]",Afterglow,"[Kosemura Akira, Nakamura Haruka]",43.126594
1,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",249789,10,"[GN1701, GN1704]",20090617,667658,[230358],York Town Girl,[Eddie Higgins Trio],242.402344
2,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",28215,10,"[GN0508, GN0501, GN2602, GN0504, GN2601]",20091217,744955,[446541],Cottage Piano,[5mg],276.400629
3,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",497515,10,"[GN1907, GN1910, GN1905, GN1912, GN1901]",20110126,1177351,[51538],The Jazzy View,[DJ Ezasscul],149.376908
4,"[향긋한, 나른한, 카페, 오후, 여유, 커피, 햇살, 재즈, 감성]",650192,10,[GN1701],20050513,130391,[193389],I M Through With Love,[Wendy Lee Taylor],2.587995


In [14]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10636 entries, 0 to 10635
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tags                10636 non-null  object 
 1   song_id             10636 non-null  int64  
 2   like_cnt            10636 non-null  int64  
 3   gnr                 10636 non-null  object 
 4   issue_date          10636 non-null  int64  
 5   album_id            10636 non-null  int64  
 6   artist_id_basket    10636 non-null  object 
 7   song_name           10636 non-null  object 
 8   artist_name_basket  10636 non-null  object 
 9   like_cnt_song       10636 non-null  float64
dtypes: float64(1), int64(4), object(5)
memory usage: 914.0+ KB


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [15]:
train_data_sample2 = train_data_origin.copy()
#train_data_sample2 = train_data_eng.copy()
train_data_sample2 = train_data_sample2.reset_index(drop=True)
#train_data_sample2 = train_data

In [16]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = song_tag_appended['tags'], vector_size = 100, 
               window = 5, min_count = 15, workers = 4, sg = 1)

w2v.wv.vectors.shape

(545, 100)

In [17]:
print(song_tag_appended.shape)

(10636, 10)


# 태그 불균형

In [18]:
def make_song_num_dict(data):
    song_ids = dict()
    song_num = dict()
    max_num = 0
    
    for i in range(len(data)):
        songs = data['song_id'][i]
        tags = data['tags'][i]
        
        for j in tags:
            if not j in song_ids:
                song_ids[j] = set(songs)
            
            else:
                song_ids[j].update(songs)
    
    for i in song_ids:
        song_num[i] = len(song_ids[i])
        
        max_num = max(song_num[i], max_num)
    
    return song_num, max_num

song_num_dict, song_num_max = make_song_num_dict(train_data_sample2)
tag_weights = {tag: np.log(song_num_max / cnt + 1) for tag, cnt in song_num_dict.items()}
tag_weights_all = {tag: np.log(len(song_tag_appended) / cnt + 1) for tag, cnt in song_num_dict.items()}
print(tag_weights)


{'햇살': 3.7495040759303713, '카페': 1.3888816846848415, '여유': 1.6564886432200994, '오후': 2.7043098814864868, '재즈': 1.8583339598507247, '나른한': 3.5603774098944156, '커피': 2.5793688329334956, '감성': 0.913203780357451, '향긋한': 4.0158197991097895, '봄': 1.3862943611198906, '설렘': 1.1556319773916537, '사랑': 0.8220041684493304, '집중': 1.9043072095519868, '아침': 3.2271746276828956, '태교': 3.5048898402192847, '낮잠': 4.079230924412053, '피아노': 1.9899567525956874, '명상': 4.079230924412053, '클래식': 2.7043098814864868, '뉴에이지': 1.478067844045871, '자장가': 2.5729302457925494, '숙면': 3.5048898402192847, '휴식': 0.9671723419055103, '창작': 4.0158197991097895, '새벽': 0.9374555430661984, '월드뮤직': 4.0158197991097895, '드라이브': 0.7632345032889003, '작업': 4.0158197991097895, '기분전환': 0.6931471805599453, '짝사랑': 2.674950252675865, '두근두근': 4.079230924412053, '고백': 2.9930111834396302, '애정': 4.079230924412053, '상사병': 4.079230924412053, '겨울': 1.6200787715688214, '메리크리스마스': 4.4790399088438795, '성탄절': 4.4790399088438795, '크리스마스': 2.566539604284

## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

(10636, 182)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim

array([[1. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 1. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 1. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 1. , 0.5],
       [0. , 0. , 0. , ..., 1. , 1. , 0.5],
       [0. , 0. , 0. , ..., 0.5, 0.5, 1. ]])

In [21]:
test_data_sample = train_data_origin.copy()
#test_data_sample = train_data_eng.copy()
test_data_sample = test_data_sample.reset_index(drop=True)
test_my_tags = test_data_sample['tags'].tolist()
test_my_songs = test_data_sample['song_id'].tolist()

In [22]:
from recommend import *

weight_mat_cv = []
weight_mat_tf = []

weight_mat_cv.append(apply_genre_weight(get_embedding(song_tag_appended, 'cv'), 0))
weight_mat_cv.append(apply_genre_weight(get_embedding(song_tag_appended, 'cv'), 1))

weight_mat_tf.append(apply_genre_weight(get_embedding(song_tag_appended, 'tf'), 0))
weight_mat_tf.append(apply_genre_weight(get_embedding(song_tag_appended, 'tf'), 1))

## 결과 (1)

In [24]:
# 1/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, False, False, 'cv', 0, 0, 0),
    (1, 'cos', True, False, False, 'cv', 0, 0, 0),
    (1, 'cos', True, False, False, 'cv', 0, 0, 1),
    (1, 'cos', False, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.291
nDCG: 0.541
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.286
nDCG: 0.522
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.286
nDCG: 0.558
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.362
nDCG: 0.655


In [25]:
# 2/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, False, 'cv', 1, 0, 0),
    (1, 'cos', True, True, False, 'cv', 0, 0, 0),
    (1, 'cos', True, True, False, 'cv', 0, 0, 1),
    (1, 'cos', True, True, False, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.302
nDCG: 0.567
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.341
nDCG: 0.597
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.348
nDCG: 0.622
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.290
nDCG: 0.527


In [26]:
# 3/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, False, 'cv', 1, 0, 1),
    (1, 'cos', False, False, False, 'tf', 0, 0, 0),
    (1, 'cos', True, False, False, 'tf', 0, 0, 0),
    (1, 'cos', True, False, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.302
nDCG: 0.559
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.296
nDCG: 0.554
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.285
nDCG: 0.527
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.312
nDCG: 0.580


In [28]:
# 4/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, False, 'tf', 0, 0, 0),
    (1, 'cos', False, True, False, 'tf', 1, 0, 0),
    (1, 'cos', True, True, False, 'tf', 0, 0, 0),
    (1, 'cos', True, True, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.309
nDCG: 0.572
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.311
nDCG: 0.578
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.305
nDCG: 0.533
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.325
nDCG: 0.604


In [29]:
# 5/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, False, 'tf', 1, 0, 0),
    (1, 'cos', True, True, False, 'tf', 1, 0, 1),
    (1, 'pea', False, False, False, 'cv', 0, 0, 0),
    (1, 'pea', True, False, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.302
nDCG: 0.536
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.316
nDCG: 0.597
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.068
nDCG: 0.168
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.066
nDCG: 0.187


In [30]:
# 6/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, False, 'cv', 0, 0, 1),
    (1, 'pea', False, True, False, 'cv', 0, 0, 0),
    (1, 'pea', False, True, False, 'cv', 1, 0, 0),
    (1, 'pea', True, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.063
nDCG: 0.179
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.121
nDCG: 0.293
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.061
nDCG: 0.172
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.128
nDCG: 0.315


In [32]:
# 7/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, False, 'cv', 0, 0, 1),
    (1, 'pea', True, True, False, 'cv', 1, 0, 0),
    (1, 'pea', True, True, False, 'cv', 1, 0, 1),
    (1, 'pea', False, False, False, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.123
nDCG: 0.287
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.066
nDCG: 0.191
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.069
nDCG: 0.190
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.065
nDCG: 0.167


In [33]:
# 8/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, False, 'tf', 0, 0, 0),
    (1, 'pea', True, False, False, 'tf', 0, 0, 1),
    (1, 'pea', False, True, False, 'tf', 0, 0, 0),
    (1, 'pea', False, True, False, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.065
nDCG: 0.184
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.060
nDCG: 0.178
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.189
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.068
nDCG: 0.180


In [34]:
# 9/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, False, 'tf', 0, 0, 0),
    (1, 'pea', True, True, False, 'tf', 0, 0, 1),
    (1, 'pea', True, True, False, 'tf', 1, 0, 0),
    (1, 'pea', True, True, False, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.193
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.074
nDCG: 0.198
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.064
nDCG: 0.185
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.193


In [36]:
# 10/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, False, False, 'cv', 0, 0, 0),
    (2, 'cos', True, False, False, 'cv', 0, 0, 0),
    (2, 'cos', True, False, False, 'cv', 0, 0, 1),
    (2, 'cos', False, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.555
nDCG: 0.718
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.560
nDCG: 0.725
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.557
nDCG: 0.720
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.586
nDCG: 0.751


In [37]:
# 11/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, False, 'cv', 1, 0, 0),
    (2, 'cos', True, True, False, 'cv', 0, 0, 0),
    (2, 'cos', True, True, False, 'cv', 0, 0, 1),
    (2, 'cos', True, True, False, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.587
nDCG: 0.750
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.591
nDCG: 0.756
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.590
nDCG: 0.754
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.593
nDCG: 0.756


In [38]:
# 12/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, False, 'cv', 1, 0, 1),
    (2, 'cos', False, False, False, 'tf', 0, 0, 0),
    (2, 'cos', True, False, False, 'tf', 0, 0, 0),
    (2, 'cos', True, False, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.590
nDCG: 0.751
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.556
nDCG: 0.710
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.562
nDCG: 0.714
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.559
nDCG: 0.716


In [40]:
# 13/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, False, 'tf', 0, 0, 0),
    (2, 'cos', False, True, False, 'tf', 1, 0, 0),
    (2, 'cos', True, True, False, 'tf', 0, 0, 0),
    (2, 'cos', True, True, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.561
nDCG: 0.717
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.584
nDCG: 0.733
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.562
nDCG: 0.721
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.567
nDCG: 0.723


In [41]:
# 14/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, False, 'tf', 1, 0, 0),
    (2, 'cos', True, True, False, 'tf', 1, 0, 1),
    (2, 'pea', False, False, False, 'cv', 0, 0, 0),
    (2, 'pea', True, False, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.588
nDCG: 0.738
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.589
nDCG: 0.740
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.076
nDCG: 0.181
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.076
nDCG: 0.181


In [42]:
# 15/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, False, 'cv', 0, 0, 1),
    (2, 'pea', False, True, False, 'cv', 0, 0, 0),
    (2, 'pea', False, True, False, 'cv', 1, 0, 0),
    (2, 'pea', True, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.076
nDCG: 0.179
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.123
nDCG: 0.259
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.085
nDCG: 0.195
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.123
nDCG: 0.260


In [44]:
# 16/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, False, 'cv', 0, 0, 1),
    (2, 'pea', True, True, False, 'cv', 1, 0, 0),
    (2, 'pea', True, True, False, 'cv', 1, 0, 1),
    (2, 'pea', False, False, False, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.121
nDCG: 0.255
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.085
nDCG: 0.194
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.085
nDCG: 0.194
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.081
nDCG: 0.204


In [45]:
# 17/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, False, 'tf', 0, 0, 0),
    (2, 'pea', True, False, False, 'tf', 0, 0, 1),
    (2, 'pea', False, True, False, 'tf', 0, 0, 0),
    (2, 'pea', False, True, False, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.081
nDCG: 0.204
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.081
nDCG: 0.205
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.097
nDCG: 0.222
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.104
nDCG: 0.225


In [46]:
# 18/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, False, 'tf', 0, 0, 0),
    (2, 'pea', True, True, False, 'tf', 0, 0, 1),
    (2, 'pea', True, True, False, 'tf', 1, 0, 0),
    (2, 'pea', True, True, False, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.097
nDCG: 0.219
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.097
nDCG: 0.222
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.105
nDCG: 0.227
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.104
nDCG: 0.225


In [48]:
# 19/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, False, True, 'cv', 0, 0, 0),
    (1, 'cos', True, False, True, 'cv', 0, 0, 0),
    (1, 'cos', True, False, True, 'cv', 0, 0, 1),
    (1, 'cos', False, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.241
nDCG: 0.426
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.237
nDCG: 0.426
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.239
nDCG: 0.431
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.302
nDCG: 0.497


In [49]:
# 20/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, True, 'cv', 1, 0, 0),
    (1, 'cos', True, True, True, 'cv', 0, 0, 0),
    (1, 'cos', True, True, True, 'cv', 0, 0, 1),
    (1, 'cos', True, True, True, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.263
nDCG: 0.453
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.291
nDCG: 0.492
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.290
nDCG: 0.490
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.259
nDCG: 0.453


In [50]:
# 21/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, True, 'cv', 1, 0, 1),
    (1, 'cos', False, False, True, 'tf', 0, 0, 0),
    (1, 'cos', True, False, True, 'tf', 0, 0, 0),
    (1, 'cos', True, False, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.258
nDCG: 0.455
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.257
nDCG: 0.443
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.253
nDCG: 0.446
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.254
nDCG: 0.446


In [52]:
# 22/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, True, 'tf', 0, 0, 0),
    (1, 'cos', False, True, True, 'tf', 1, 0, 0),
    (1, 'cos', True, True, True, 'tf', 0, 0, 0),
    (1, 'cos', True, True, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.275
nDCG: 0.475
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.272
nDCG: 0.471
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.272
nDCG: 0.478
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.272
nDCG: 0.478


In [53]:
# 23/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, True, 'tf', 1, 0, 0),
    (1, 'cos', True, True, True, 'tf', 1, 0, 1),
    (1, 'pea', False, False, True, 'cv', 0, 0, 0),
    (1, 'pea', True, False, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.267
nDCG: 0.470
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.266
nDCG: 0.470
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.022
nDCG: 0.067
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.022
nDCG: 0.068


In [54]:
# 24/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, True, 'cv', 0, 0, 1),
    (1, 'pea', False, True, True, 'cv', 0, 0, 0),
    (1, 'pea', False, True, True, 'cv', 1, 0, 0),
    (1, 'pea', True, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.022
nDCG: 0.073
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.031
nDCG: 0.077
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.019
nDCG: 0.075
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.031
nDCG: 0.078


In [56]:
# 25/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, True, 'cv', 0, 0, 1),
    (1, 'pea', True, True, True, 'cv', 1, 0, 0),
    (1, 'pea', True, True, True, 'cv', 1, 0, 1),
    (1, 'pea', False, False, True, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.029
nDCG: 0.077
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.019
nDCG: 0.067
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.023
nDCG: 0.083
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.068


In [57]:
# 26/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, True, 'tf', 0, 0, 0),
    (1, 'pea', True, False, True, 'tf', 0, 0, 1),
    (1, 'pea', False, True, True, 'tf', 0, 0, 0),
    (1, 'pea', False, True, True, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.022
nDCG: 0.066
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.016
nDCG: 0.061
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.027
nDCG: 0.075
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.071


In [58]:
# 27/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, True, 'tf', 0, 0, 0),
    (1, 'pea', True, True, True, 'tf', 0, 0, 1),
    (1, 'pea', True, True, True, 'tf', 1, 0, 0),
    (1, 'pea', True, True, True, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.025
nDCG: 0.074
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.023
nDCG: 0.072
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.023
nDCG: 0.073
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.025
nDCG: 0.073


In [60]:
# 28/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, False, True, 'cv', 0, 0, 0),
    (2, 'cos', True, False, True, 'cv', 0, 0, 0),
    (2, 'cos', True, False, True, 'cv', 0, 0, 1),
    (2, 'cos', False, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.467
nDCG: 0.687
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.469
nDCG: 0.688
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.469
nDCG: 0.685
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.508
nDCG: 0.719


In [61]:
# 29/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, True, 'cv', 1, 0, 0),
    (2, 'cos', True, True, True, 'cv', 0, 0, 0),
    (2, 'cos', True, True, True, 'cv', 0, 0, 1),
    (2, 'cos', True, True, True, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.497
nDCG: 0.704
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.511
nDCG: 0.722
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.512
nDCG: 0.721
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.498
nDCG: 0.705


In [62]:
# 30/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, True, 'cv', 1, 0, 1),
    (2, 'cos', False, False, True, 'tf', 0, 0, 0),
    (2, 'cos', True, False, True, 'tf', 0, 0, 0),
    (2, 'cos', True, False, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.502
nDCG: 0.708
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.489
nDCG: 0.697
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.491
nDCG: 0.697
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.491
nDCG: 0.696


In [64]:
# 31/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, True, 'tf', 0, 0, 0),
    (2, 'cos', False, True, True, 'tf', 1, 0, 0),
    (2, 'cos', True, True, True, 'tf', 0, 0, 0),
    (2, 'cos', True, True, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.505
nDCG: 0.715
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.505
nDCG: 0.710
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.507
nDCG: 0.717
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.507
nDCG: 0.712


In [65]:
# 32/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, True, 'tf', 1, 0, 0),
    (2, 'cos', True, True, True, 'tf', 1, 0, 1),
    (2, 'pea', False, False, True, 'cv', 0, 0, 0),
    (2, 'pea', True, False, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.507
nDCG: 0.713
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.507
nDCG: 0.713
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.007
nDCG: 0.010
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.007
nDCG: 0.010


In [66]:
# 33/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, True, 'cv', 0, 0, 1),
    (2, 'pea', False, True, True, 'cv', 0, 0, 0),
    (2, 'pea', False, True, True, 'cv', 1, 0, 0),
    (2, 'pea', True, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.007
nDCG: 0.010
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.011
nDCG: 0.017
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013


In [68]:
# 34/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, True, 'cv', 0, 0, 1),
    (2, 'pea', True, True, True, 'cv', 1, 0, 0),
    (2, 'pea', True, True, True, 'cv', 1, 0, 1),
    (2, 'pea', False, False, True, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.011
nDCG: 0.017
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.011
nDCG: 0.017
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.005
nDCG: 0.007


In [69]:
# 35/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, True, 'tf', 0, 0, 0),
    (2, 'pea', True, False, True, 'tf', 0, 0, 1),
    (2, 'pea', False, True, True, 'tf', 0, 0, 0),
    (2, 'pea', False, True, True, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.005
nDCG: 0.007
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.005
nDCG: 0.007
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017


In [70]:
# 36/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, True, 'tf', 0, 0, 0),
    (2, 'pea', True, True, True, 'tf', 0, 0, 1),
    (2, 'pea', True, True, True, 'tf', 1, 0, 0),
    (2, 'pea', True, True, True, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017


# 태그 자연어 전처리

## 한국어 전처리

In [72]:
import konlpy
from konlpy.tag import Okt

train_data_sample = train_data_origin.copy()

okt = Okt()
for i in range(len(train_data_sample)):
    preprocessed_tags = []
    for tag in train_data_sample['tags'][i]:
        normalized_tag = okt.normalize(tag)
        pos_tagging = okt.pos(normalized_tag)
        for word, pos in pos_tagging:
            if pos == 'Noun' or pos == 'Verb' or pos == 'Adjective':
                preprocessed_tags.append(word)
            elif pos == 'Alpha':
                preprocessed_tags.append(word.lower())
    if len(preprocessed_tags) == 0:
        train_data_sample['tags'][i] = preprocessed_tags
    else:
        preprocessed_tags = list(dict.fromkeys(preprocessed_tags))
        train_data_sample['tags'][i] = preprocessed_tags
train_data_sample.head(20)
preprocessed_plist = train_data_sample.copy()

C:\Users\Hyun\AppData\Local\Temp\ipykernel_15164\12101270.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_sample['tags'][i] = preprocessed_tags


In [73]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)

train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})

C:\Users\Hyun\AppData\Local\Temp\ipykernel_15164\299166450.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_sample['tags'].iloc[i] = list(train_dict[song])


In [74]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = song_tag_appended['tags'], vector_size = 100, 
               window = 5, min_count = 15, workers = 4, sg = 1)

w2v.wv.vectors.shape

(536, 100)

In [75]:
song_num_dict, song_num_max = make_song_num_dict(preprocessed_plist)
tag_weights = {tag: np.log(song_num_max / cnt + 1) for tag, cnt in song_num_dict.items()}
tag_weights_all = {tag: np.log(len(song_tag_appended) / cnt + 1) for tag, cnt in song_num_dict.items()}
print(tag_weights)

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

{'햇살': 3.882054595499302, '카페': 1.3677958684076557, '여유': 1.7675432232868373, '오후': 2.831329217262381, '재즈': 1.9322929173634826, '나른한': 3.692304952936964, '커피': 2.7052490788224985, '감성': 0.7124801708972748, '향': 4.149069461911347, '긋한': 4.149069461911347, '봄': 1.4660801974845397, '설렘': 1.2504417461336004, '사랑': 0.9002148750163199, '집중': 2.020787313427954, '아침': 2.577997072854012, '태교': 3.6366110255681794, '낮잠': 4.212621303146474, '피아노': 2.108014914123892, '명상': 4.212621303146474, '클래식': 2.725047777470781, '뉴에이지': 1.5842694584938488, '자장가': 2.6987478084398964, '숙면': 3.6366110255681794, '휴식': 1.0226498566951772, '창작': 4.149069461911347, '새벽': 0.979058326979858, '월드뮤직': 4.149069461911347, '드라이브': 0.8378673868758725, '작업': 4.149069461911347, '기분': 0.6931471805599453, '전환': 0.7007479265513843, '짝사랑': 2.801714676996845, '고백': 3.122173371953489, '애정': 4.212621303146474, '상사병': 4.212621303146474, '겨울': 1.654184399439723, '메리크리스마스': 4.613138355637268, '성탄절': 4.613138355637268, '크리스마스': 2.692294

In [76]:
#test_data_sample = train_data[500:1000].copy()
test_data_sample = preprocessed_plist.copy()
test_data_sample = test_data_sample.reset_index(drop=True)
test_my_tags = test_data_sample['tags'].tolist()
test_my_songs = test_data_sample['song_id'].tolist()

weight_mat_cv = []
weight_mat_tf = []

weight_mat_cv.append(apply_genre_weight(get_embedding(song_tag_appended, 'cv'), 0))
weight_mat_cv.append(apply_genre_weight(get_embedding(song_tag_appended, 'cv'), 1))

weight_mat_tf.append(apply_genre_weight(get_embedding(song_tag_appended, 'tf'), 0))
weight_mat_tf.append(apply_genre_weight(get_embedding(song_tag_appended, 'tf'), 1))

## 결과 (2)

In [78]:
# 1/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, False, False, 'cv', 0, 0, 0),
    (1, 'cos', True, False, False, 'cv', 0, 0, 0),
    (1, 'cos', True, False, False, 'cv', 0, 0, 1),
    (1, 'cos', False, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.287
nDCG: 0.529
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.281
nDCG: 0.517
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.282
nDCG: 0.528
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.355
nDCG: 0.623


In [79]:
# 2/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, False, 'cv', 1, 0, 0),
    (1, 'cos', True, True, False, 'cv', 0, 0, 0),
    (1, 'cos', True, True, False, 'cv', 0, 0, 1),
    (1, 'cos', True, True, False, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.303
nDCG: 0.556
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.354
nDCG: 0.633
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.343
nDCG: 0.614
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.293
nDCG: 0.542


In [80]:
# 3/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, False, 'cv', 1, 0, 1),
    (1, 'cos', False, False, False, 'tf', 0, 0, 0),
    (1, 'cos', True, False, False, 'tf', 0, 0, 0),
    (1, 'cos', True, False, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.288
nDCG: 0.554
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.302
nDCG: 0.549
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.302
nDCG: 0.527
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.294
nDCG: 0.559


In [82]:
# 4/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, False, 'tf', 0, 0, 0),
    (1, 'cos', False, True, False, 'tf', 1, 0, 0),
    (1, 'cos', True, True, False, 'tf', 0, 0, 0),
    (1, 'cos', True, True, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.319
nDCG: 0.582
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.309
nDCG: 0.560
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.310
nDCG: 0.561
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.313
nDCG: 0.575


In [83]:
# 5/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, False, 'tf', 1, 0, 0),
    (1, 'cos', True, True, False, 'tf', 1, 0, 1),
    (1, 'pea', False, False, False, 'cv', 0, 0, 0),
    (1, 'pea', True, False, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.305
nDCG: 0.564
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.299
nDCG: 0.549
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.063
nDCG: 0.161
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.064
nDCG: 0.177


In [84]:
# 6/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, False, 'cv', 0, 0, 1),
    (1, 'pea', False, True, False, 'cv', 0, 0, 0),
    (1, 'pea', False, True, False, 'cv', 1, 0, 0),
    (1, 'pea', True, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.060
nDCG: 0.158
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.119
nDCG: 0.284
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.074
nDCG: 0.184
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.134
nDCG: 0.303


In [86]:
# 7/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, False, 'cv', 0, 0, 1),
    (1, 'pea', True, True, False, 'cv', 1, 0, 0),
    (1, 'pea', True, True, False, 'cv', 1, 0, 1),
    (1, 'pea', False, False, False, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.116
nDCG: 0.274
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.068
nDCG: 0.185
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.062
nDCG: 0.163
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.055
nDCG: 0.157


In [87]:
# 8/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, False, 'tf', 0, 0, 0),
    (1, 'pea', True, False, False, 'tf', 0, 0, 1),
    (1, 'pea', False, True, False, 'tf', 0, 0, 0),
    (1, 'pea', False, True, False, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.063
nDCG: 0.179
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.054
nDCG: 0.160
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.076
nDCG: 0.189
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.071
nDCG: 0.184


In [88]:
# 9/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, False, 'tf', 0, 0, 0),
    (1, 'pea', True, True, False, 'tf', 0, 0, 1),
    (1, 'pea', True, True, False, 'tf', 1, 0, 0),
    (1, 'pea', True, True, False, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.080
nDCG: 0.210
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.079
nDCG: 0.209
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.067
nDCG: 0.177
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.072
nDCG: 0.164


In [90]:
# 10/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, False, False, 'cv', 0, 0, 0),
    (2, 'cos', True, False, False, 'cv', 0, 0, 0),
    (2, 'cos', True, False, False, 'cv', 0, 0, 1),
    (2, 'cos', False, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.561
nDCG: 0.729
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.563
nDCG: 0.727
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.564
nDCG: 0.732
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.589
nDCG: 0.758


In [91]:
# 11/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, False, 'cv', 1, 0, 0),
    (2, 'cos', True, True, False, 'cv', 0, 0, 0),
    (2, 'cos', True, True, False, 'cv', 0, 0, 1),
    (2, 'cos', True, True, False, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.597
nDCG: 0.760
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.595
nDCG: 0.758
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.594
nDCG: 0.764
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.598
nDCG: 0.761


In [92]:
# 12/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, False, 'cv', 1, 0, 1),
    (2, 'cos', False, False, False, 'tf', 0, 0, 0),
    (2, 'cos', True, False, False, 'tf', 0, 0, 0),
    (2, 'cos', True, False, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.599
nDCG: 0.762
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.563
nDCG: 0.723
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.562
nDCG: 0.715
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.569
nDCG: 0.726


In [94]:
# 13/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, False, 'tf', 0, 0, 0),
    (2, 'cos', False, True, False, 'tf', 1, 0, 0),
    (2, 'cos', True, True, False, 'tf', 0, 0, 0),
    (2, 'cos', True, True, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.564
nDCG: 0.722
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.591
nDCG: 0.740
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.567
nDCG: 0.723
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.565
nDCG: 0.722


In [95]:
# 14/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, False, 'tf', 1, 0, 0),
    (2, 'cos', True, True, False, 'tf', 1, 0, 1),
    (2, 'pea', False, False, False, 'cv', 0, 0, 0),
    (2, 'pea', True, False, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.590
nDCG: 0.739
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.590
nDCG: 0.738
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.182
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.078
nDCG: 0.184


In [96]:
# 15/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, False, 'cv', 0, 0, 1),
    (2, 'pea', False, True, False, 'cv', 0, 0, 0),
    (2, 'pea', False, True, False, 'cv', 1, 0, 0),
    (2, 'pea', True, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.078
nDCG: 0.185
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.124
nDCG: 0.264
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.088
nDCG: 0.199
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.125
nDCG: 0.265


In [98]:
# 16/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, False, 'cv', 0, 0, 1),
    (2, 'pea', True, True, False, 'cv', 1, 0, 0),
    (2, 'pea', True, True, False, 'cv', 1, 0, 1),
    (2, 'pea', False, False, False, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.125
nDCG: 0.266
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.089
nDCG: 0.201
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.090
nDCG: 0.204
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.084
nDCG: 0.210


In [99]:
# 17/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, False, 'tf', 0, 0, 0),
    (2, 'pea', True, False, False, 'tf', 0, 0, 1),
    (2, 'pea', False, True, False, 'tf', 0, 0, 0),
    (2, 'pea', False, True, False, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.086
nDCG: 0.212
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.086
nDCG: 0.212
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.096
nDCG: 0.217
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.106
nDCG: 0.231


In [100]:
# 18/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, False, 'tf', 0, 0, 0),
    (2, 'pea', True, True, False, 'tf', 0, 0, 1),
    (2, 'pea', True, True, False, 'tf', 1, 0, 0),
    (2, 'pea', True, True, False, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.097
nDCG: 0.217
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.096
nDCG: 0.217
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.108
nDCG: 0.234
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.109
nDCG: 0.235


In [102]:
# 19/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, False, True, 'cv', 0, 0, 0),
    (1, 'cos', True, False, True, 'cv', 0, 0, 0),
    (1, 'cos', True, False, True, 'cv', 0, 0, 1),
    (1, 'cos', False, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.251
nDCG: 0.437
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.250
nDCG: 0.439
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.249
nDCG: 0.435
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.310
nDCG: 0.518


In [103]:
# 20/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, True, 'cv', 1, 0, 0),
    (1, 'cos', True, True, True, 'cv', 0, 0, 0),
    (1, 'cos', True, True, True, 'cv', 0, 0, 1),
    (1, 'cos', True, True, True, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.274
nDCG: 0.468
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.305
nDCG: 0.511
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.305
nDCG: 0.504
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.274
nDCG: 0.470


In [104]:
# 21/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, True, 'cv', 1, 0, 1),
    (1, 'cos', False, False, True, 'tf', 0, 0, 0),
    (1, 'cos', True, False, True, 'tf', 0, 0, 0),
    (1, 'cos', True, False, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.274
nDCG: 0.467
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.266
nDCG: 0.453
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.264
nDCG: 0.452
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.267
nDCG: 0.451


In [106]:
# 22/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, True, 'tf', 0, 0, 0),
    (1, 'cos', False, True, True, 'tf', 1, 0, 0),
    (1, 'cos', True, True, True, 'tf', 0, 0, 0),
    (1, 'cos', True, True, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.285
nDCG: 0.477
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.281
nDCG: 0.475
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.282
nDCG: 0.478
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.284
nDCG: 0.481


In [107]:
# 23/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, True, 'tf', 1, 0, 0),
    (1, 'cos', True, True, True, 'tf', 1, 0, 1),
    (1, 'pea', False, False, True, 'cv', 0, 0, 0),
    (1, 'pea', True, False, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.280
nDCG: 0.478
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.281
nDCG: 0.477
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.071
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.025
nDCG: 0.067


In [108]:
# 24/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, True, 'cv', 0, 0, 1),
    (1, 'pea', False, True, True, 'cv', 0, 0, 0),
    (1, 'pea', False, True, True, 'cv', 1, 0, 0),
    (1, 'pea', True, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.026
nDCG: 0.072
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.033
nDCG: 0.093
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.026
nDCG: 0.077
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.029
nDCG: 0.080


In [110]:
# 25/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, True, 'cv', 0, 0, 1),
    (1, 'pea', True, True, True, 'cv', 1, 0, 0),
    (1, 'pea', True, True, True, 'cv', 1, 0, 1),
    (1, 'pea', False, False, True, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.032
nDCG: 0.089
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.028
nDCG: 0.070
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.028
nDCG: 0.083
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.082


In [111]:
# 26/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, True, 'tf', 0, 0, 0),
    (1, 'pea', True, False, True, 'tf', 0, 0, 1),
    (1, 'pea', False, True, True, 'tf', 0, 0, 0),
    (1, 'pea', False, True, True, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.026
nDCG: 0.076
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.025
nDCG: 0.078
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.079
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.026
nDCG: 0.079


In [112]:
# 27/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, True, 'tf', 0, 0, 0),
    (1, 'pea', True, True, True, 'tf', 0, 0, 1),
    (1, 'pea', True, True, True, 'tf', 1, 0, 0),
    (1, 'pea', True, True, True, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.028
nDCG: 0.085
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.028
nDCG: 0.085
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.026
nDCG: 0.073
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.072


In [114]:
# 28/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, False, True, 'cv', 0, 0, 0),
    (2, 'cos', True, False, True, 'cv', 0, 0, 0),
    (2, 'cos', True, False, True, 'cv', 0, 0, 1),
    (2, 'cos', False, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.474
nDCG: 0.695
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.474
nDCG: 0.694
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.475
nDCG: 0.696
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.510
nDCG: 0.722


In [115]:
# 29/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, True, 'cv', 1, 0, 0),
    (2, 'cos', True, True, True, 'cv', 0, 0, 0),
    (2, 'cos', True, True, True, 'cv', 0, 0, 1),
    (2, 'cos', True, True, True, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.502
nDCG: 0.714
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.511
nDCG: 0.727
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.513
nDCG: 0.729
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.502
nDCG: 0.714


In [116]:
# 30/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, True, 'cv', 1, 0, 1),
    (2, 'cos', False, False, True, 'tf', 0, 0, 0),
    (2, 'cos', True, False, True, 'tf', 0, 0, 0),
    (2, 'cos', True, False, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.504
nDCG: 0.714
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.493
nDCG: 0.703
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.492
nDCG: 0.702
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.496
nDCG: 0.707


In [118]:
# 31/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, True, 'tf', 0, 0, 0),
    (2, 'cos', False, True, True, 'tf', 1, 0, 0),
    (2, 'cos', True, True, True, 'tf', 0, 0, 0),
    (2, 'cos', True, True, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.509
nDCG: 0.717
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.508
nDCG: 0.716
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.513
nDCG: 0.721
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.512
nDCG: 0.723


In [119]:
# 32/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, True, 'tf', 1, 0, 0),
    (2, 'cos', True, True, True, 'tf', 1, 0, 1),
    (2, 'pea', False, False, True, 'cv', 0, 0, 0),
    (2, 'pea', True, False, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.506
nDCG: 0.714
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.510
nDCG: 0.721
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.008
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.008
nDCG: 0.013


In [120]:
# 33/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, True, 'cv', 0, 0, 1),
    (2, 'pea', False, True, True, 'cv', 0, 0, 0),
    (2, 'pea', False, True, True, 'cv', 1, 0, 0),
    (2, 'pea', True, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.008
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.020
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013


In [122]:
# 34/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, True, 'cv', 0, 0, 1),
    (2, 'pea', True, True, True, 'cv', 1, 0, 0),
    (2, 'pea', True, True, True, 'cv', 1, 0, 1),
    (2, 'pea', False, False, True, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.020
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.020
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.006
nDCG: 0.010


In [123]:
# 35/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, True, 'tf', 0, 0, 0),
    (2, 'pea', True, False, True, 'tf', 0, 0, 1),
    (2, 'pea', False, True, True, 'tf', 0, 0, 0),
    (2, 'pea', False, True, True, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.006
nDCG: 0.010
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.006
nDCG: 0.010
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017


In [124]:
# 36/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, True, 'tf', 0, 0, 0),
    (2, 'pea', True, True, True, 'tf', 0, 0, 1),
    (2, 'pea', True, True, True, 'tf', 1, 0, 0),
    (2, 'pea', True, True, True, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017


# 한국어 -> 영어 변환
- 한국어 전처리에 어려움이 있어서, 영어로 모두 변환 후 처리

In [126]:
'''
from googletrans import Translator
import time

def trans_to_eng(tags):
    time.sleep(1)
    translator = Translator()
    translated = translator.translate(tags, src = 'ko', dest = 'en')
    return translated.text

for i in range(len(train_data_sample)):
    eng_tags = [trans_to_eng(tag) for tag in train_data_sample['tags'][i]]
    train_data_sample['tags'][i] = eng_tags

train_data_sample.to_json('../Datasets/train_eng1.json', orient='records')
train_data_sample.head(30)

    
#train_data.head()
#eng_tags = [trans_to_eng(tag) for tag in train_data['tags'][1]]
#print("Kor: ", train_data['tags'][1])
#print("Eng: ", eng_tags)
'''

'\nfrom googletrans import Translator\nimport time\n\ndef trans_to_eng(tags):\n    time.sleep(1)\n    translator = Translator()\n    translated = translator.translate(tags, src = \'ko\', dest = \'en\')\n    return translated.text\n\nfor i in range(len(train_data_sample)):\n    eng_tags = [trans_to_eng(tag) for tag in train_data_sample[\'tags\'][i]]\n    train_data_sample[\'tags\'][i] = eng_tags\n\ntrain_data_sample.to_json(\'../Datasets/train_eng1.json\', orient=\'records\')\ntrain_data_sample.head(30)\n\n    \n#train_data.head()\n#eng_tags = [trans_to_eng(tag) for tag in train_data[\'tags\'][1]]\n#print("Kor: ", train_data[\'tags\'][1])\n#print("Eng: ", eng_tags)\n'

In [127]:
with open('../Datasets/train_datas_eng.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

train_data_eng = pd.DataFrame(json_data)

train_data_eng.shape
#print(train_data_eng.loc[train_data_indices])

(10000, 3)

In [128]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('popular') # nltk 라이브러리 사용을 위해 다운해야 함

with open('../Datasets/train_datas_eng.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

train_data_eng = pd.DataFrame(json_data)
train_data_eng = train_data_eng.loc[train_data_indices]
train_data_eng = train_data_eng.reset_index(drop=True)

print(train_data_eng['tags'][3])
pattern = re.compile('[^a-zA-Z0-9]')#특수문자나 공백을 띄어쓰기로 처리할지는 이후 테스트
idx = 0
for tags in train_data_eng['tags']:
    eng_tags = []
    for tag in tags:
        temp_tags = re.sub(pattern, ' ', tag).lower().split()
        for temp_tag in temp_tags:
            eng_tags.append(temp_tag)
    train_data_eng['tags'][idx] = eng_tags
    idx += 1
print(train_data_eng['tags'][3])

#Stopwords
stops = set(stopwords.words('english'))

for i in range(len(train_data_eng)):
    eng_tags = [tag for tag in train_data_eng['tags'][i] if not tag in stops]
    if train_data_eng['tags'][i] != eng_tags:
        print(train_data_eng['tags'][i])
        print(eng_tags)
    train_data_eng['tags'][i] = eng_tags
    
#Stemming
stemmer = nltk.stem.SnowballStemmer('english')
for i in range(len(train_data_eng)):
    eng_tags = [stemmer.stem(tag) for tag in train_data_eng['tags'][i]]
    train_data_eng['tags'][i] = eng_tags

#중복 제거
for i in range(len(train_data_eng)):
    eng_tags = list(dict.fromkeys(train_data_eng['tags'][i]))
    train_data_eng['tags'][i] = eng_tags
    
#한 글자 제거
for i in range(len(train_data_eng)):
    eng_tags = [tag for tag in train_data_eng['tags'][i] if len(tag) > 1]
    train_data_eng['tags'][i] = eng_tags


# '록' 또는 '락' 이 'lock' 으로 번역되는 문제가 있어서, 'rock' 으로 일괄적으로 수정
pattern = re.compile(r'\block\b')
for i in range(len(train_data_eng)):
    eng_tags = [re.sub(pattern, 'rock', tag) for tag in train_data_eng['tags'][i]]
    train_data_eng['tags'][i] = eng_tags



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\Hyun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\Hyun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\Hyun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\Hyun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\Hyun\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |   

['rest', 'Creation', 'cafe', 'dawn', 'World Music', 'drive', 'jazz', 'Emotion', 'work', 'Mood']
['rest', 'creation', 'cafe', 'dawn', 'world', 'music', 'drive', 'jazz', 'emotion', 'work', 'mood']
['summer', 'the', 'rainy', 'season', 'idol', 'drive', 'travel', 'mood', 'exciting', 'song', 'stress', 'recorded', 'song', 'exciting']
['summer', 'rainy', 'season', 'idol', 'drive', 'travel', 'mood', 'exciting', 'song', 'stress', 'recorded', 'song', 'exciting']
['lounge', 'what', 'is', 'your', 'brother', 's', 'song', 'drive', 'emotional', 'itaewon', 'mood', 'i', 'only', 'know', 'you']
['lounge', 'brother', 'song', 'drive', 'emotional', 'itaewon', 'mood', 'know']
['windless', 'emotion', 'the', 'songs', 'are', 'the', 'songs', 'summer', 'night', 'idol', 'drive', 'cool', 'love', 'stress', 'exciting']
['windless', 'emotion', 'songs', 'songs', 'summer', 'night', 'idol', 'drive', 'cool', 'love', 'stress', 'exciting']
['dreamstation', 'live', 'ponau', 'welovevocaledm', 'zurboy', 'spectrum', 'edm', 'with

C:\Users\Hyun\AppData\Local\Temp\ipykernel_15164\3447819155.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_eng['tags'][i] = eng_tags
C:\Users\Hyun\AppData\Local\Temp\ipykernel_15164\3447819155.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_eng['tags'][i] = eng_tags
C:\Users\Hyun\AppData\Local\Temp\ipykernel_15164\3447819155.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_eng['tags'][i] 

In [129]:
train_data_sample = train_data_eng.copy()
train_data_sample = train_data_sample.reset_index(drop=True)
train_data_sample.head(20)

,tags,song_id,like_cnt
0,"[sunlight, cafe, spare, afternoon, jazz, drows...","[314824, 249789, 28215, 497515, 650192, 182422...",10
1,"[spring, flutter, love]","[396832, 84145, 610632, 512162, 505209, 411774...",4
2,"[concentr, morn, prenat, nap, piano, medit, cl...","[418643, 85532, 33582, 361338, 404014, 193549,...",41
3,"[rest, creation, cafe, dawn, world, music, dri...","[494436, 196322, 191118, 430859, 332650, 70067...",3355
4,"[crush, pound, love, confess, flutter, affect,...","[312406, 506781, 328452, 425497, 380114, 2284,...",225
5,"[winter, merri, christma, pop]","[387859, 357047, 131659, 613741, 318487, 33289...",3
6,"[heal, rest, mood]","[394183, 397924, 311707, 625914, 244640, 53548...",0
7,"[consol, night, dawn]","[666827, 457284, 389128, 23957, 488376, 677289...",6
8,"[electronica, danc, edmfloor, edm, billow]","[31529, 618572, 257867, 328496, 372879, 588590...",0
9,"[2017, person, tast, old, song]","[402779, 173943, 624242, 557503, 521517, 45338...",0


In [130]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)

In [131]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)

In [132]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

C:\Users\Hyun\AppData\Local\Temp\ipykernel_15164\2746467678.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_sample['tags'].iloc[i] = list(train_dict[song])


In [133]:
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,like_cnt,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,like_cnt_song
0,"[cafe, spare, drowsi, aromen, jazz, coffe, emo...",314824,10,"[GN1912, GN1904, GN1901]",20070914,534654,"[408062, 408064]",Afterglow,"[Kosemura Akira, Nakamura Haruka]",43.126594
1,"[cafe, spare, drowsi, aromen, jazz, coffe, emo...",249789,10,"[GN1701, GN1704]",20090617,667658,[230358],York Town Girl,[Eddie Higgins Trio],242.402344
2,"[cafe, spare, drowsi, aromen, jazz, coffe, emo...",28215,10,"[GN0508, GN0501, GN2602, GN0504, GN2601]",20091217,744955,[446541],Cottage Piano,[5mg],276.400629
3,"[cafe, spare, drowsi, aromen, jazz, coffe, emo...",497515,10,"[GN1907, GN1910, GN1905, GN1912, GN1901]",20110126,1177351,[51538],The Jazzy View,[DJ Ezasscul],149.376908
4,"[cafe, spare, drowsi, aromen, jazz, coffe, emo...",650192,10,[GN1701],20050513,130391,[193389],I M Through With Love,[Wendy Lee Taylor],2.587995


In [134]:
from gensim.models.word2vec import Word2Vec

train_data_sample2 = train_data_eng.copy()
train_data_sample2 = train_data_sample2.reset_index(drop=True)

w2v = Word2Vec(sentences = song_tag_appended['tags'], vector_size = 100, 
               window = 5, min_count = 15, workers = 4, sg = 1)

w2v.wv.vectors.shape

(467, 100)

In [135]:
song_num_dict, song_num_max = make_song_num_dict(train_data_sample2)
tag_weights = {tag: np.log(song_num_max / cnt + 1) for tag, cnt in song_num_dict.items()}
tag_weights_all = {tag: np.log(len(song_tag_appended) / cnt + 1) for tag, cnt in song_num_dict.items()}
print(tag_weights)

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x : (' ').join(x))

{'sunlight': 3.7578723256008875, 'cafe': 0.8880687086500749, 'spare': 1.6634282963306397, 'afternoon': 2.7123077913548514, 'jazz': 1.824740149957914, 'drowsi': 3.09922357343648, 'coffe': 2.587290712228522, 'emot': 0.7157918770236508, 'aromen': 4.024234996035074, 'spring': 1.3478836925633537, 'flutter': 1.1615108890914296, 'love': 0.8048168654110363, 'concentr': 1.9116047364638402, 'morn': 1.824740149957914, 'prenat': 3.51320244003865, 'nap': 4.087655574071304, 'piano': 1.9973587108913553, 'medit': 4.087655574071304, 'classic': 2.606913556560569, 'new': 1.3243688542728522, 'age': 1.4846889231888072, 'lullabi': 2.580847943529335, 'deep': 3.51320244003865, 'sleep': 2.1802156630348932, 'rest': 0.9432218675400159, 'creation': 4.024234996035074, 'dawn': 0.9016968303202262, 'world': 3.0326816104671, 'music': 0.89972270164881, 'drive': 0.7678181629565582, 'work': 1.9488412698641722, 'mood': 0.6931471805599453, 'crush': 2.682931141006878, 'pound': 4.087655574071304, 'confess': 3.001152341023330

In [136]:
#test_data_sample = train_data[500:1000].copy()
test_data_sample = train_data_eng.copy()
test_data_sample = test_data_sample.reset_index(drop=True)
test_my_tags = test_data_sample['tags'].tolist()
test_my_songs = test_data_sample['song_id'].tolist()

weight_mat_cv = []
weight_mat_tf = []

weight_mat_cv.append(apply_genre_weight(get_embedding(song_tag_appended, 'cv'), 0))
weight_mat_cv.append(apply_genre_weight(get_embedding(song_tag_appended, 'cv'), 1))

weight_mat_tf.append(apply_genre_weight(get_embedding(song_tag_appended, 'tf'), 0))
weight_mat_tf.append(apply_genre_weight(get_embedding(song_tag_appended, 'tf'), 1))

## 결과 (3)

In [138]:
# 1/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, False, False, 'cv', 0, 0, 0),
    (1, 'cos', True, False, False, 'cv', 0, 0, 0),
    (1, 'cos', True, False, False, 'cv', 0, 0, 1),
    (1, 'cos', False, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.276
nDCG: 0.535
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.285
nDCG: 0.555
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.299
nDCG: 0.565
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.339
nDCG: 0.593


In [139]:
# 2/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, False, 'cv', 1, 0, 0),
    (1, 'cos', True, True, False, 'cv', 0, 0, 0),
    (1, 'cos', True, True, False, 'cv', 0, 0, 1),
    (1, 'cos', True, True, False, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.290
nDCG: 0.560
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.343
nDCG: 0.626
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.341
nDCG: 0.618
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.298
nDCG: 0.557


In [140]:
# 3/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, False, 'cv', 1, 0, 1),
    (1, 'cos', False, False, False, 'tf', 0, 0, 0),
    (1, 'cos', True, False, False, 'tf', 0, 0, 0),
    (1, 'cos', True, False, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.294
nDCG: 0.562
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.286
nDCG: 0.551
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.286
nDCG: 0.544
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.304
nDCG: 0.582


In [142]:
# 4/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, False, 'tf', 0, 0, 0),
    (1, 'cos', False, True, False, 'tf', 1, 0, 0),
    (1, 'cos', True, True, False, 'tf', 0, 0, 0),
    (1, 'cos', True, True, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.306
nDCG: 0.567
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.300
nDCG: 0.565
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.299
nDCG: 0.555
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.306
nDCG: 0.567


In [143]:
# 5/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, False, 'tf', 1, 0, 0),
    (1, 'cos', True, True, False, 'tf', 1, 0, 1),
    (1, 'pea', False, False, False, 'cv', 0, 0, 0),
    (1, 'pea', True, False, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.311
nDCG: 0.574
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.315
nDCG: 0.584
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.069
nDCG: 0.175
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.064
nDCG: 0.167


In [144]:
# 6/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, False, 'cv', 0, 0, 1),
    (1, 'pea', False, True, False, 'cv', 0, 0, 0),
    (1, 'pea', False, True, False, 'cv', 1, 0, 0),
    (1, 'pea', True, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.062
nDCG: 0.166
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.123
nDCG: 0.290
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.078
nDCG: 0.194
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.125
nDCG: 0.300


In [146]:
# 7/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, False, 'cv', 0, 0, 1),
    (1, 'pea', True, True, False, 'cv', 1, 0, 0),
    (1, 'pea', True, True, False, 'cv', 1, 0, 1),
    (1, 'pea', False, False, False, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.112
nDCG: 0.287
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.072
nDCG: 0.181
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.067
nDCG: 0.182
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.063
nDCG: 0.171


In [147]:
# 8/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, False, 'tf', 0, 0, 0),
    (1, 'pea', True, False, False, 'tf', 0, 0, 1),
    (1, 'pea', False, True, False, 'tf', 0, 0, 0),
    (1, 'pea', False, True, False, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.064
nDCG: 0.168
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.068
nDCG: 0.182
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.081
nDCG: 0.197
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.083
nDCG: 0.205


In [148]:
# 9/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, False, 'tf', 0, 0, 0),
    (1, 'pea', True, True, False, 'tf', 0, 0, 1),
    (1, 'pea', True, True, False, 'tf', 1, 0, 0),
    (1, 'pea', True, True, False, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.073
nDCG: 0.185
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.069
nDCG: 0.192
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.079
nDCG: 0.203
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.191


In [150]:
# 10/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, False, False, 'cv', 0, 0, 0),
    (2, 'cos', True, False, False, 'cv', 0, 0, 0),
    (2, 'cos', True, False, False, 'cv', 0, 0, 1),
    (2, 'cos', False, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.556
nDCG: 0.727
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.560
nDCG: 0.725
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.563
nDCG: 0.732
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.591
nDCG: 0.749


In [151]:
# 11/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, False, 'cv', 1, 0, 0),
    (2, 'cos', True, True, False, 'cv', 0, 0, 0),
    (2, 'cos', True, True, False, 'cv', 0, 0, 1),
    (2, 'cos', True, True, False, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.584
nDCG: 0.750
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.593
nDCG: 0.751
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.593
nDCG: 0.755
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.587
nDCG: 0.752


In [152]:
# 12/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, False, 'cv', 1, 0, 1),
    (2, 'cos', False, False, False, 'tf', 0, 0, 0),
    (2, 'cos', True, False, False, 'tf', 0, 0, 0),
    (2, 'cos', True, False, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.588
nDCG: 0.755
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.553
nDCG: 0.706
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.558
nDCG: 0.710
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.560
nDCG: 0.714


In [154]:
# 13/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, False, 'tf', 0, 0, 0),
    (2, 'cos', False, True, False, 'tf', 1, 0, 0),
    (2, 'cos', True, True, False, 'tf', 0, 0, 0),
    (2, 'cos', True, True, False, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.562
nDCG: 0.714
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.579
nDCG: 0.732
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.566
nDCG: 0.718
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.566
nDCG: 0.718


In [155]:
# 14/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, False, 'tf', 1, 0, 0),
    (2, 'cos', True, True, False, 'tf', 1, 0, 1),
    (2, 'pea', False, False, False, 'cv', 0, 0, 0),
    (2, 'pea', True, False, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.585
nDCG: 0.736
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.583
nDCG: 0.733
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.184
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.186


In [156]:
# 15/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, False, 'cv', 0, 0, 1),
    (2, 'pea', False, True, False, 'cv', 0, 0, 0),
    (2, 'pea', False, True, False, 'cv', 1, 0, 0),
    (2, 'pea', True, True, False, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.077
nDCG: 0.187
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.120
nDCG: 0.254
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.090
nDCG: 0.203
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.120
nDCG: 0.254


In [158]:
# 16/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, False, 'cv', 0, 0, 1),
    (2, 'pea', True, True, False, 'cv', 1, 0, 0),
    (2, 'pea', True, True, False, 'cv', 1, 0, 1),
    (2, 'pea', False, False, False, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.120
nDCG: 0.256
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.089
nDCG: 0.201
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.091
nDCG: 0.204
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.083
nDCG: 0.206


In [159]:
# 17/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, False, 'tf', 0, 0, 0),
    (2, 'pea', True, False, False, 'tf', 0, 0, 1),
    (2, 'pea', False, True, False, 'tf', 0, 0, 0),
    (2, 'pea', False, True, False, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.084
nDCG: 0.206
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.084
nDCG: 0.206
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.094
nDCG: 0.215
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.107
nDCG: 0.232


In [160]:
# 18/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, False, 'tf', 0, 0, 0),
    (2, 'pea', True, True, False, 'tf', 0, 0, 1),
    (2, 'pea', True, True, False, 'tf', 1, 0, 0),
    (2, 'pea', True, True, False, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.093
nDCG: 0.214
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.092
nDCG: 0.210
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.106
nDCG: 0.231
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : False, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.107
nDCG: 0.231


In [162]:
# 19/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, False, True, 'cv', 0, 0, 0),
    (1, 'cos', True, False, True, 'cv', 0, 0, 0),
    (1, 'cos', True, False, True, 'cv', 0, 0, 1),
    (1, 'cos', False, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.250
nDCG: 0.440
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.246
nDCG: 0.436
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.244
nDCG: 0.434
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.302
nDCG: 0.506


In [163]:
# 20/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, True, 'cv', 1, 0, 0),
    (1, 'cos', True, True, True, 'cv', 0, 0, 0),
    (1, 'cos', True, True, True, 'cv', 0, 0, 1),
    (1, 'cos', True, True, True, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.269
nDCG: 0.464
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.295
nDCG: 0.508
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.299
nDCG: 0.511
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.266
nDCG: 0.462


In [164]:
# 21/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, True, 'cv', 1, 0, 1),
    (1, 'cos', False, False, True, 'tf', 0, 0, 0),
    (1, 'cos', True, False, True, 'tf', 0, 0, 0),
    (1, 'cos', True, False, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.266
nDCG: 0.456
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.266
nDCG: 0.454
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.261
nDCG: 0.451
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.261
nDCG: 0.448


In [166]:
# 22/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', False, True, True, 'tf', 0, 0, 0),
    (1, 'cos', False, True, True, 'tf', 1, 0, 0),
    (1, 'cos', True, True, True, 'tf', 0, 0, 0),
    (1, 'cos', True, True, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.283
nDCG: 0.475
Recommend Model : 1, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.279
nDCG: 0.471
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.275
nDCG: 0.475
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.278
nDCG: 0.474


In [167]:
# 23/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'cos', True, True, True, 'tf', 1, 0, 0),
    (1, 'cos', True, True, True, 'tf', 1, 0, 1),
    (1, 'pea', False, False, True, 'cv', 0, 0, 0),
    (1, 'pea', True, False, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.273
nDCG: 0.472
Recommend Model : 1, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.274
nDCG: 0.468
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.022
nDCG: 0.066
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.021
nDCG: 0.060


In [168]:
# 24/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, True, 'cv', 0, 0, 1),
    (1, 'pea', False, True, True, 'cv', 0, 0, 0),
    (1, 'pea', False, True, True, 'cv', 1, 0, 0),
    (1, 'pea', True, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.022
nDCG: 0.064
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.032
nDCG: 0.085
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.072
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.031
nDCG: 0.087


In [170]:
# 25/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, True, 'cv', 0, 0, 1),
    (1, 'pea', True, True, True, 'cv', 1, 0, 0),
    (1, 'pea', True, True, True, 'cv', 1, 0, 1),
    (1, 'pea', False, False, True, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.032
nDCG: 0.091
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.024
nDCG: 0.077
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.023
nDCG: 0.075
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.023
nDCG: 0.073


In [171]:
# 26/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, False, True, 'tf', 0, 0, 0),
    (1, 'pea', True, False, True, 'tf', 0, 0, 1),
    (1, 'pea', False, True, True, 'tf', 0, 0, 0),
    (1, 'pea', False, True, True, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.021
nDCG: 0.075
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.022
nDCG: 0.075
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.029
nDCG: 0.079
Recommend Model : 1, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.021
nDCG: 0.080


In [172]:
# 27/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (1, 'pea', True, True, True, 'tf', 0, 0, 0),
    (1, 'pea', True, True, True, 'tf', 0, 0, 1),
    (1, 'pea', True, True, True, 'tf', 1, 0, 0),
    (1, 'pea', True, True, True, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.028
nDCG: 0.083
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.026
nDCG: 0.088
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.021
nDCG: 0.080
Recommend Model : 1, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.021
nDCG: 0.077


In [174]:
# 28/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, False, True, 'cv', 0, 0, 0),
    (2, 'cos', True, False, True, 'cv', 0, 0, 0),
    (2, 'cos', True, False, True, 'cv', 0, 0, 1),
    (2, 'cos', False, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.472
nDCG: 0.697
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.473
nDCG: 0.694
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.474
nDCG: 0.698
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.511
nDCG: 0.720


In [175]:
# 29/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, True, 'cv', 1, 0, 0),
    (2, 'cos', True, True, True, 'cv', 0, 0, 0),
    (2, 'cos', True, True, True, 'cv', 0, 0, 1),
    (2, 'cos', True, True, True, 'cv', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.498
nDCG: 0.712
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.512
nDCG: 0.720
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.511
nDCG: 0.718
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.498
nDCG: 0.711


In [176]:
# 30/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, True, 'cv', 1, 0, 1),
    (2, 'cos', False, False, True, 'tf', 0, 0, 0),
    (2, 'cos', True, False, True, 'tf', 0, 0, 0),
    (2, 'cos', True, False, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.499
nDCG: 0.709
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.489
nDCG: 0.699
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.490
nDCG: 0.697
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.490
nDCG: 0.699


In [178]:
# 31/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', False, True, True, 'tf', 0, 0, 0),
    (2, 'cos', False, True, True, 'tf', 1, 0, 0),
    (2, 'cos', True, True, True, 'tf', 0, 0, 0),
    (2, 'cos', True, True, True, 'tf', 0, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.507
nDCG: 0.717
Recommend Model : 2, Simi_mode : cos, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.508
nDCG: 0.714
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.507
nDCG: 0.717
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.507
nDCG: 0.716


In [179]:
# 32/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'cos', True, True, True, 'tf', 1, 0, 0),
    (2, 'cos', True, True, True, 'tf', 1, 0, 1),
    (2, 'pea', False, False, True, 'cv', 0, 0, 0),
    (2, 'pea', True, False, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.508
nDCG: 0.713
Recommend Model : 2, Simi_mode : cos, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.509
nDCG: 0.718
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.008
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.008
nDCG: 0.013


In [180]:
# 33/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, True, 'cv', 0, 0, 1),
    (2, 'pea', False, True, True, 'cv', 0, 0, 0),
    (2, 'pea', False, True, True, 'cv', 1, 0, 0),
    (2, 'pea', True, True, True, 'cv', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.008
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.020
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013


In [182]:
# 34/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, True, 'cv', 0, 0, 1),
    (2, 'pea', True, True, True, 'cv', 1, 0, 0),
    (2, 'pea', True, True, True, 'cv', 1, 0, 1),
    (2, 'pea', False, False, True, 'tf', 0, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.010
nDCG: 0.013
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.020
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : cv, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.020
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.006
nDCG: 0.010


In [183]:
# 35/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, False, True, 'tf', 0, 0, 0),
    (2, 'pea', True, False, True, 'tf', 0, 0, 1),
    (2, 'pea', False, True, True, 'tf', 0, 0, 0),
    (2, 'pea', False, True, True, 'tf', 1, 0, 0)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.006
nDCG: 0.010
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : False, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.006
nDCG: 0.010
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : False, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017


In [184]:
# 36/36
params_list = [
    #recnum(1, 2) / simi_mode(cos, pea, jac) / 태그 / 장르 / 좋아요 / mode(cv/tf) / 장르 가중치 모드(0/1) / 좋아요 가중치 모드(0/1) / 태그 가중치 모드(0/1)
    (2, 'pea', True, True, True, 'tf', 0, 0, 0),
    (2, 'pea', True, True, True, 'tf', 0, 0, 1),
    (2, 'pea', True, True, True, 'tf', 1, 0, 0),
    (2, 'pea', True, True, True, 'tf', 1, 0, 1)
]

with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(process_fuc, [(param[0], test_my_songs, test_my_tags, song_tag_appended, param[1], param[2], param[3], param[4], 
                                         param[5], param[6], param[7], param[8], w2v, weight_mat_cv, weight_mat_tf, tag_weights, tag_weights_all) for param in params_list])
    
for result in results:
    print(result)

Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 0, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.014
nDCG: 0.023
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 0, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017
Recommend Model : 2, Simi_mode : pea, Tag : True, Genre : True, Like : True, Mode : tf, Genre Weight Mode : 1, Tag Weight Mode : 1, Like Weight Mode : 0
MAP@K (K=10): 0.012
nDCG: 0.017
